Instalar a biblioteca `ultralytics`.

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

Criar diretório `segmentacao` e adicionar as pastas `model` e `videos` copiadas do Google Drive.

O arquivo `best.pt` da pasta `model` foi gerado pelo treinamento da rede utilizando este outro Colab:

https://colab.research.google.com/drive/1fyGCk_hMf-kYg62ASMaOle7XChxBypXY

Já o arquivo `data.yaml`, nessa mesma pasta, foi gerado utilizando o Roboflow, como explicando no Colab desse link.

Os vídeos da pasta `videos` não foram utilizadas nesse treinamento e serão úteis para testar o modelo treinado.

In [ ]:
!mkdir /content/segmentacao
!mkdir /content/segmentacao/model/
!mkdir /content/segmentacao/videos/
!mkdir /content/segmentacao/out/

import gdown

#https://drive.google.com/file/d/15kE3MNhMV0_xXv6rlPpwNTBELnN3s0Vg/view?usp=sharing
file_id = '15kE3MNhMV0_xXv6rlPpwNTBELnN3s0Vg'
gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', 'best.pt', quiet=False)
!mv best.pt /content/segmentacao/model/

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#https://drive.google.com/file/d/1BAy8QVuHlUByOWzx_bVOTEopRIssU-Lr/view?usp=sharing
file_id = '1BAy8QVuHlUByOWzx_bVOTEopRIssU-Lr'
gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', 'data.yaml', quiet=False)
!mv data.yaml /content/segmentacao/model/

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#https://drive.google.com/file/d/1_ig46XMu-IgwEdfPmL8zjK35X3CW9eFb/view?usp=sharing
file_id = '1_ig46XMu-IgwEdfPmL8zjK35X3CW9eFb'
gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', 'estrada01.mp4', quiet=False)
!mv estrada01.mp4 /content/segmentacao/videos/


#https://drive.google.com/file/d/1xEG57VWTMY9AWFAAx2WZZgqsPl42BA7t/view?usp=sharing
file_id = '1xEG57VWTMY9AWFAAx2WZZgqsPl42BA7t'
gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', 'estrada05.mp4', quiet=False)
!mv estrada05.mp4 /content/segmentacao/videos/

Downloading...
From: https://drive.google.com/uc?export=download&id=15kE3MNhMV0_xXv6rlPpwNTBELnN3s0Vg
To: /content/best.pt
100%|██████████| 54.8M/54.8M [00:02<00:00, 22.3MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1BAy8QVuHlUByOWzx_bVOTEopRIssU-Lr
To: /content/data.yaml
100%|██████████| 334/334 [00:00<00:00, 1.37MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1_ig46XMu-IgwEdfPmL8zjK35X3CW9eFb
To: /content/estrada01.mp4
100%|██████████| 12.1M/12.1M [00:00<00:00, 29.2MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1xEG57VWTMY9AWFAAx2WZZgqsPl42BA7t
To: /content/estrada05.mp4
100%|██████████| 3.04M/3.04M [00:00<00:00, 20.7MB/s]


In [ ]:
def load_classes():
    #with open('/data.yaml', 'r') as config_file:
    with open('/content/segmentacao/model/data.yaml', 'r') as config_file:
        config_data = config_file.read()

    class_names_start = config_data.find("names: [") + len("names: [")
    class_names_end = config_data.find("]", class_names_start)
    class_names_str = config_data[class_names_start:class_names_end]
    class_names = [name.strip().strip("'") for name in class_names_str.split(",")]

    return class_names

nomeClasses = load_classes()

FileNotFoundError: [Errno 2] No such file or directory: '/content/segmentacao/model/data.yaml'

In [ ]:
import random

def class_colors(names):
    return {name: (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for name in names}

color = class_colors(nomeClasses)

In [ ]:
import cv2

dev = '/content/segmentacao/videos/estrada05.mp4'
cap = cv2.VideoCapture(dev)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)

#result = cv2.VideoWriter(path + "file00.mp4", cv2.VideoWriter_fourcc(*'MJPG'), 5, size)
#result = cv2.VideoWriter(path + "output.mp4", -1, 5, size)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
result = cv2.VideoWriter('/content/segmentacao/out/output_video.avi', fourcc, 5, size)

In [ ]:
from ultralytics import YOLO
import numpy as np

# Carregar o modelo treinado YOLOv8
#model = YOLO('/content/segmentacao/model/best.pt')

class YOLOSegmentation:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def detect(self, img):
        # Get img shape

        height, width, channels = img.shape

        results = self.model.predict(source=img.copy(), save=False, save_txt=False)
        result = results[0]
        segmentation_contours_idx = []
        #for seg in result.masks.segments:
        for seg in result.masks.xyn:
            # contours
            seg[:, 0] *= width
            seg[:, 1] *= height
            segment = np.array(seg, dtype=np.int32)
            segmentation_contours_idx.append(segment)

        bboxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
        # Get class ids
        class_ids = np.array(result.boxes.cls.cpu(), dtype="int")
        # Get scores
        scores = np.array(result.boxes.conf.cpu(), dtype="float").round(2)
        return bboxes, class_ids, segmentation_contours_idx, scores


model = YOLOSegmentation('/content/segmentacao/model/best.pt')

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
      break

    bboxes, classes, segmentations, scores = model.detect(frame)

    for bbox, class_id, seg, score in zip(bboxes, classes, segmentations, scores):
      (x, y, x2, y2) = bbox
      className = nomeClasses[class_id]

      cv2.polylines(frame, [seg], True, color[className], 3)

      result.write(frame)


0: 480x640 1 rural_road, 2 skys, 2 vegetations, 1747.1ms
Speed: 16.5ms preprocess, 1747.1ms inference, 56.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 rural_road, 2 skys, 2 vegetations, 1534.6ms
Speed: 3.8ms preprocess, 1534.6ms inference, 15.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 rural_road, 2 skys, 2 vegetations, 1542.7ms
Speed: 3.5ms preprocess, 1542.7ms inference, 18.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 rural_road, 2 skys, 2 vegetations, 2357.9ms
Speed: 3.4ms preprocess, 2357.9ms inference, 23.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 rural_road, 2 skys, 2 vegetations, 1996.2ms
Speed: 5.0ms preprocess, 1996.2ms inference, 18.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 rural_road, 2 skys, 2 vegetations, 1563.8ms
Speed: 3.2ms preprocess, 1563.8ms inference, 14.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 rural_road, 2 skys, 2 vegetations, 15

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 640):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

In [ ]:
show_video('/content/segmentacao/videos/estrada05.mp4')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
show_video('/content/segmentacao/out/output_video.avi')

Output hidden; open in https://colab.research.google.com to view.

Se o video não funcionar, faça o download para visualizá-lo.

In [ ]:
from google.colab import files
files.download('/content/segmentacao/out/output_video.avi')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Até aqui

